In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

random_state = 0
np.random.seed(random_state)
pd.set_option("display.max_columns", False)

In [ ]:
!wget -r -N -c -np https://physionet.org/files/eicu-crd-demo/2.0.1/ &> /dev/null

In [ ]:
!ls physionet.org/files/eicu-crd-demo/2.0.1/

In [ ]:
patients = pd.read_csv("physionet.org/files/eicu-crd-demo/2.0.1/patient.csv.gz")

In [ ]:
patients.head()

In [ ]:
patients = patients[[
    "patientunitstayid", "patienthealthsystemstayid", "gender", "age", 
    "ethnicity", "apacheadmissiondx", "admissionheight", "hospitaladmitsource", 
    "hospitaldischargestatus", "unitadmitsource", "unitvisitnumber", 
    "admissionweight"
]]

# Note that the age column has a value "> 89" to mitigate re-identification risks. This entry prevents the conversion to a number. We'll map this to 90.

In [ ]:
patients["age"].unique()

In [ ]:
patients.loc[patients["age"] == "> 89", "age"] = "90"
patients["age"] = pd.to_numeric(patients["age"])

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111)
patients["hospitaldischargestatus"].hist(bins=2, xlabelsize=16)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_ylabel("Counts", fontsize=16)
ax.set_xlabel("Hospital discharge status")

# It is very common when looking at clinical outcomes, such as mortality, that the outcome is very unevenly distributed. This makes prediction problems really challenging!

In [ ]:
no_alive = len(patients[patients['hospitaldischargestatus'] == 'Alive'])
no_died = len(patients[patients["hospitaldischargestatus"] == "Expired"])
no_total = no_alive + no_died

In [ ]:
print(f"{no_alive} patients who left the hospital alive ({100*no_alive/no_total:.2f}%) and {no_died} who died in hospital ({100*no_died/no_total:.2f}%).")

# Let's see if we can predict who will leave the hospital alive! We'll only use the first unit stay during a hospitalization.

Task 1: Keep only the first stay for each patient

In [ ]:
len(patients)

# Since we only keep the first stay for each patient, the unitvisitnumber is not required anymore

In [ ]:
patients = patients.drop(["patientunitstayid", "unitvisitnumber"], axis=1)

# Convert some features to categorical and look at the histograms

In [ ]:
categorical_features = ["gender", "ethnicity", "apacheadmissiondx", "hospitaladmitsource", "hospitaldischargestatus", "unitadmitsource"]
for feat in categorical_features:
    patients.loc[:, feat] = patients.loc[:, feat].astype("category")

In [ ]:
#fig, ax = plt.subplots(nrows=len(categorical_features), ncols=1, figsize=(18, 36))
for feat in categorical_features:
    fig = plt.figure(figsize=(12,8))
    ax = fig.add_subplot(111)
    patients[feat].value_counts().plot(kind="bar", ax=ax, fontsize=18)
    ax.set_xlabel(f"{feat}", fontsize=18)
    ax.set_ylabel("Counts", fontsize=18)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    plt.tight_layout()


In [ ]:
for feat in ["admissionheight", "admissionweight", "age"]:
    fig = plt.figure(figsize=(12,8))
    ax = fig.add_subplot(111)
    patients[feat].hist(ax=ax, bins=40)
    ax.set_xlabel(f"{feat}", fontsize=18)
    ax.set_ylabel("Counts", fontsize=18)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    plt.tight_layout()

Let us generate some features for our prediction task

In [ ]:
patients["outcome"] = np.where(patients["hospitaldischargestatus"] == "Alive", 0, 1)
patients["admitsource"] = np.where(patients["hospitaladmitsource"] == "Emergency Department", 1, 0)
patients["gender"] = np.where(patients["gender"] == "Female", 1, 0)
patients = patients.drop(["hospitaldischargestatus", "hospitaladmitsource", "unitadmitsource", "apacheadmissiondx", "ethnicity"], axis=1)

In [ ]:
patients.head()

In [ ]:
patients.dtypes

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
len(patients["patienthealthsystemstayid"].unique())

# Task 2: Generate a train/test split with sizes 80% / 20%

In [ ]:
train_patients = patients[patients["patienthealthsystemstayid"].isin(train_pat_ids)]
test_patients = patients[patients["patienthealthsystemstayid"].isin(test_pat_ids)]

# It is very important to carefully set up your test problem. It is unlikely that the model will perform well on the test data set if the distribution of the outcome (mortality) is very different from the outcome on the training data.

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
test_patients["outcome"].value_counts().plot(kind="bar", ax=ax, fontsize=18)
ax.set_xlabel(f"{feat}", fontsize=18)
ax.set_ylabel("Counts", fontsize=18)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.tight_layout()

In [ ]:
no_alive_test = len(test_patients[test_patients["outcome"] == 0])
no_died_test = len(test_patients[test_patients["outcome"] == 1])
no_total_test = no_alive_test + no_died_test

In [ ]:
print(f"{no_alive_test} patients who left the hospital alive ({100*no_alive_test/no_total_test:.2f}%) and {no_died} who died in hospital ({100*no_died_test/no_total_test:.2f}%).")

In [ ]:
train_patients.head()

In [ ]:
train_patients.dtypes

In [ ]:
x_train = train_patients[["gender", "age", "admissionheight", "admissionweight"]].to_numpy()
y_train = train_patients[["outcome"]].to_numpy()

# Which model should we use?

- How can we evaluate whether a complex model gives us a benefit?
- What is the simplest model you can think of?

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [ ]:
model.fit(x_train, y_train)

# Oops, we got ahead of ourselves... Not all rows and columns contain valid data

Missingness is a problem we encounter often when working with electronic health records

In [ ]:
train_patients.loc[train_patients.isna().any(axis=1), :]

# Task 3: Identify all columns that have missing values

The simplest solution is to do mean or median imputation

In [ ]:
means = train_patients.mean()

In [ ]:
train_patients = train_patients.fillna(means)

In [ ]:
train_patients.isna().any(axis=0)

In [ ]:
x_train = train_patients[["gender", "age", "admissionheight", "admissionweight"]].to_numpy()
y_train = np.squeeze(train_patients[["outcome"]].to_numpy())

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(x_train, y_train)

In [ ]:
y_hat = model.predict(x_train)

In [ ]:
score = accuracy_score(y_train, y_hat)

In [ ]:
print(score)

# We have an accuracy above 90% !
Let's look at predictions...

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111)
ax.hist(y_hat, bins=[0, 0.5, 1])
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_ylabel("Counts")
ax.set_xlabel("Predictions")

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111)
conf_mat = confusion_matrix(y_train, y_hat)
ConfusionMatrixDisplay(confusion_matrix=conf_mat).plot(ax=ax)

In [ ]:
from sklearn.metrics import average_precision_score

In [ ]:
average_precision_score(y_train, y_hat)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
models = {
    "SVM": SVC(),
    "KNN5": KNeighborsClassifier(n_neighbors=5),
    "KNN10": KNeighborsClassifier(n_neighbors=10),
    "KNN15": KNeighborsClassifier(n_neighbors=15),
    "KNN20": KNeighborsClassifier(n_neighbors=20),
    "GBC": GradientBoostingClassifier(),
    "mlp_0": MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000),
    "mlp_1": MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000),
    "mlp_2": MLPClassifier(hidden_layer_sizes=(100, 50, 30), max_iter=1000)
}

In [ ]:
for model_name, model in models.items():
    results = cross_validate(model, scaler.transform(x_train), y_train, cv=5, scoring="average_precision")
    print(model_name, results["test_score"], np.median(results["test_score"]))

In [ ]:
models["GBC"].fit(scaler.transform(x_train), y_train)

In [ ]:
y_hat = models["GBC"].predict(scaler.transform(x_train))

In [ ]:
y_hat

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111)
conf_mat = confusion_matrix(y_train, y_hat)
ConfusionMatrixDisplay(confusion_matrix=conf_mat).plot(ax=ax)

# How can we make better predictions?
We do have more data available...

# Task 4: Predictions on the test set

In [ ]:
from sklearn.metrics import average_precision_score

In [ ]:
average_precision_score(y_test, y_hat)

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111)
conf_mat = confusion_matrix(y_test, y_hat)
ConfusionMatrixDisplay(confusion_matrix=conf_mat).plot(ax=ax)

# Solution to tasks:



In [ ]:
# Task 1:

# patients = patients.loc[patients.groupby("patienthealthsystemstayid")["unitvisitnumber"].idxmin().reset_index(drop=True), :]

In [ ]:
# Task 2:

# train_pat_ids, test_pat_ids = train_test_split(patients["patienthealthsystemstayid"].unique(), test_size=0.2)

In [ ]:
# Task 3:

# train_patients.isna().any(axis=0)

In [ ]:
# Task 4.1:

# x_test = test_patients[["gender", "age", "admissionheight", "admissionweight"]].fillna(means).to_numpy()
# y_test = np.squeeze(test_patients[["outcome"]].to_numpy())

In [ ]:
# Task 4.2

# y_hat = models["GBC"].predict(scaler.transform(x_test))